In [1]:
import nibabel as nib
import numpy as np
from joblib import Parallel, delayed
import multiprocessing

import os
import subprocess



import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import scipy.io as scio

import numpy as np
import os
from glob import glob
import nibabel as nb
import nibabel
#from pylab import *
import scipy.signal
from nilearn.masking import apply_mask
from nilearn.masking import unmask
import nilearn.signal
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"   # to hide os warning about h5py
import numpy as np
from glob import glob
import nibabel as nib
import re
from sklearn.decomposition import PCA
import itertools
import nilearn.masking
from nilearn import image
import scipy.ndimage
from nilearn.masking import apply_mask, unmask
import nipype.interfaces.fsl as fsl
import pickle
from joblib import Parallel, delayed
import nipype.pipeline.engine as pe          # pypeline engine 
import nipype.interfaces.spm as spm          # spm
#import nipype.interfaces.matlab as mlab
#mlab.MatlabCommand.set_default_paths('/home/miv/bhanot/soft/Ants/bin') # Set paths for Ants
#mlab.MatlabCommand.set_default_paths('/home/miv/bhanot/spm12')  # Set paths for spm
from nilearn.masking import compute_multi_epi_mask, compute_epi_mask, new_img_like
import scipy
import nipype.interfaces.ants as ants
#import skimage.exposure as ex
from nipype.interfaces.nipy import ComputeMask

import sys
sys.path.append('libs/')
#from time_series import *

from scipy.signal import butter, lfilter

import datetime



def BuildImagePath(pathDatasink,shorterPath,intFolder,printResults):
    print('\nCreating array of image paths...')    
    os.chdir(pathDatasink)

    arrayPath = !{'ls'}
    print('Array size: %d items.' % len(arrayPath))
    #print(array)

    arrayPathImage = arrayPath
    for index in range (len(arrayPath)):

        #print(index)
        #print(array[0])

        cmd = 'ls ' + arrayPath[index] + intFolder
        image = !{cmd}
        #print(image)


        arrayPathImage[index] = pathDatasink + '/' + arrayPathImage[index] + intFolder + '/' + image[0]
        
        if shorterPath:
            arrayPathImage[index] = pathDatasink + '/' + image[0]
        
        if printResults:
            print(arrayPathImage[index])

    if printResults:
        print(arrayPathImage)
    
    print('Array with paths made!!!\n\n')    
    
    return arrayPathImage


def PrintArray(arrayPrint):
    print('\nPrinting array...')
    print('Array length: %d' % (len(arrayPrint)))
    for index in range (len(arrayPrint)):
        print(arrayPrint[index])
    print('Array printing done!!!\n')
    
    
def SortArrayByRefArray(arrayUnsorted, arrayRef, printResults):
    arraySorted = [None]*len(arrayRef)

    for index in range (len(arrayRef)):
        indexMatch = [i for i, s in enumerate(arrayUnsorted) if arrayRef[index] in s]
        #print(indexMatch)

        if indexMatch==[]:
            print('\nAttention: Subject %s was not found!!!\n' % arrayRef[index])
            print('\nSetting path for the subject %s as None!!!\n' % arrayRef[index])                    
        
        elif (len(indexMatch) > 1):
            print('\nProblem of matching!!! Multiple indexes found!!!\n')
            break

        else:
            if printResults:
                print(index)
                print(indexMatch)
            arraySorted[index] = arrayUnsorted[indexMatch[0]]

    if printResults:    
        print(arraySorted)
    
    return arraySorted


def TapTerminalCommand(commonCmd, arraySubj, arrayPathInput, arrayPathRef, arrayPathTrans, sufix, transFormat, inverseTrans, posEraseEnd, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arraySubj)
    cmdArray = [None]*len(arraySubj)
    
    
    for indexSubject in range (len(arraySubj)):
        pathOutputImage = arrayPathInput[indexSubject][:-posEraseEnd] + sufix + '_'+ arraySubj[indexSubject] + '.nii'
        if transFormat == '.nii':
            cmd = commonCmd + ' --input ' + arrayPathInput[indexSubject] + ' --output ' + pathOutputImage + ' --reference-image ' + arrayPathRef[indexSubject] + ' --transform ' + arrayPathTrans[indexSubject]
        
        elif transFormat == '.mat':
            cmd = commonCmd + ' --input ' + arrayPathInput[indexSubject] + ' --output ' + pathOutputImage + ' --reference-image ' + arrayPathRef[indexSubject] + ' --transform ' + '[' +  arrayPathTrans[indexSubject] + ',' + str(inverseTrans) + ']'
        
        arrayPathOutput[indexSubject] = pathOutputImage
        cmdArray[indexSubject] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    return arrayPathOutput, cmdArray


def TapFSLTerminalCommand(commonCmd, arraySubj, arrayPathInput, mathOper, suffix, pathOutput, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arraySubj)
    cmdArray = [None]*len(arraySubj)
    
    
    for indexSubject in range (len(arraySubj)):
        
        pathOutputImage = pathOutput + '/' + suffix + '_'+ arraySubj[indexSubject] + '.nii.gz'
        
        cmd = commonCmd + ' ' + arrayPathInput[indexSubject] + ' ' + mathOper + ' ' + pathOutputImage
        
        arrayPathOutput[indexSubject] = pathOutputImage
        cmdArray[indexSubject] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    return arrayPathOutput, cmdArray



def TapANTsRegisteringTerminalCommand(commonCmd, arraySubj, arrayPathInputRef, arrayPathInputToBeRegistered, pathOutput, posEraseEnd, printCmd, executeCmd):
    arrayPathOutputImage = [None]*len(arraySubj)
    arrayPathOutputTransfMat = [None]*len(arraySubj)
    arrayPathOutputTransfComposite = [None]*len(arraySubj)
    arrayPathOutputTransfInvComposite = [None]*len(arraySubj)
    cmdArray = [None]*len(arraySubj)
    
    
    for indexSubject in range (len(arraySubj)):
        
        arrayPathInputRefHeadTail = os.path.split(arrayPathInputRef[indexSubject])
        arrayPathInputToBeRegisteredHeadTail = os.path.split(arrayPathInputToBeRegistered[indexSubject])
        
        pathOutputReg = pathOutput + '/' + 'outputANTsReg_' + arraySubj[indexSubject] + '_' + arrayPathInputToBeRegisteredHeadTail[1][:-posEraseEnd] + '_2_' + arrayPathInputRefHeadTail[1][:-posEraseEnd] + '_'
        
        cmd = commonCmd + ' --initial-moving-transform [' + arrayPathInputRef[indexSubject] + ',' + arrayPathInputToBeRegistered[indexSubject] + ', 1]' + ' --output [ ' + pathOutputReg + ' , ' + pathOutputReg + 'warpedImage.nii' + ']' + ' --save-state ' + pathOutputReg + 'trans.mat' + ' --transform Affine[ 2.0 ] --metric Mattes[ ' + arrayPathInputRef[indexSubject] + ' , ' + arrayPathInputToBeRegistered[indexSubject] + ', 1, 32, Random, 0.05 ]' + ' --convergence [ 1500x200, 1e-08, 20 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1' + ' --transform SyN[ 0.25, 3.0, 0.0 ] --metric Mattes[ ' + arrayPathInputRef[indexSubject] + ' , ' + arrayPathInputToBeRegistered[indexSubject] + ', 1, 32 ]' + ' --convergence [ 100x50x30, 1e-09, 20 ] --smoothing-sigmas 2.0x1.0x0.0vox --shrink-factors 3x2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --winsorize-image-intensities [ 0.0, 1.0 ] --write-composite-transform 1'
        
        arrayPathOutputImage[indexSubject] = pathOutputReg + 'warpedImage.nii'
        arrayPathOutputTransfMat[indexSubject] = pathOutputReg + 'trans.mat'
        arrayPathOutputTransfComposite[indexSubject] = pathOutputReg + 'Composite.h5'
        arrayPathOutputTransfInvComposite[indexSubject] = pathOutputReg + 'InverseComposite.h5'
        
        cmdArray[indexSubject] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    return arrayPathOutputImage, arrayPathOutputTransfMat, arrayPathOutputTransfComposite, arrayPathOutputTransfInvComposite, cmdArray



def TapCopyTerminalCommand(commonCmd, arraySubj, arrayPathInput, newName, pathOutput, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arraySubj)
    cmdArray = [None]*len(arraySubj)
    
    
    for indexSubject in range (len(arraySubj)):
        
        pathOutputImage = pathOutput + '/' + newName + '_'+ arraySubj[indexSubject] + '.nii.gz'
        
        cmd = commonCmd + ' ' + arrayPathInput[indexSubject] + ' ' + pathOutputImage
        
        arrayPathOutput[indexSubject] = pathOutputImage
        cmdArray[indexSubject] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    return arrayPathOutput, cmdArray


def TapCopyTerminalCommandReg3DAtlas2TractFolder(commonCmd, arrayBaseFileToBeCopied, posEraseExt, pathBaseOutput, arrayOrderSubjOutput, nameRegAtlasTracFolder, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arrayOrderSubjOutput)
    cmdArray = [None]*len(arrayOrderSubjOutput)
    
    for indexFolder in range (len(arrayOrderSubjOutput)):
        
        pathOutputImage = pathBaseOutput + '/' + arrayOrderSubjOutput[indexFolder] + '/' + nameRegAtlasTracFolder
        
        cmd = commonCmd + ' ' + arrayBaseFileToBeCopied[indexFolder] + ' ' + pathOutputImage
        
        arrayPathOutput[indexFolder] = pathOutputImage
        cmdArray[indexFolder] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    
    return arrayPathOutput,cmdArray



def TapFSLSwapDimTerminalCommand(optionCmd, arrayInputImages, posEraseExt, suffixInputImages, suffixOutput, extSaveFile, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arrayInputImages)
    cmdArray = [None]*len(arrayInputImages)
    
    for indexImage in range (len(arrayInputImages)):
        
        pathOutputImage = arrayInputImages[indexImage][:-posEraseExt] + suffixInputImages[:-posEraseExt] + suffixOutput + extSaveFile
        
        cmd = 'fslswapdim ' + arrayInputImages[indexImage][:-posEraseExt] + suffixInputImages + ' ' + optionCmd + ' ' + pathOutputImage
        
        arrayPathOutput[indexImage] = pathOutputImage
        cmdArray[indexImage] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    
    return arrayPathOutput,cmdArray


def TapMrTransformTerminalCommand(optionCmd, arrayInputImages, posEraseExt, transformPath, suffixOutput, extSaveFile, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arrayInputImages)
    cmdArray = [None]*len(arrayInputImages)
    
    for indexImage in range (len(arrayInputImages)):
        
        pathOutputImage = arrayInputImages[indexImage][:-posEraseExt] + suffixOutput + extSaveFile
        
        cmd = 'mrtransform ' + optionCmd + ' ' + transformPath + ' ' + arrayInputImages[indexImage] + ' ' + pathOutputImage
        
        arrayPathOutput[indexImage] = pathOutputImage
        cmdArray[indexImage] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    
    return arrayPathOutput,cmdArray
    
    
def TapTck2ConnectomeTerminalCommand(arrayOptions, pathMainFolder, arrayPathInputFolders, arrayTcks, nameRegAtlasImage, outputPrefix, arrayOutputSuffix, extSaveFile, printCmd, executeCmd):
    
    arrayPathOutputConnectomes = [None]*(len(arrayOptions)*len(arrayTcks)*len(arrayPathInputFolders))
    cmdArray = [None]*(len(arrayOptions)*len(arrayTcks)*len(arrayPathInputFolders))
    
    for indexFolder in range (len(arrayPathInputFolders)):
        for indexOption in range (len(arrayOptions)):
            for indexTck in range (len(arrayTcks)):
                
                pathOutputConnectome = pathMainFolder + '/' + arrayPathInputFolders[indexFolder] + '/' + outputPrefix + arrayTcks[indexTck][:-4] + arrayOutputSuffix[indexOption] + extSaveFile
                
                cmd = 'tck2connectome ' + arrayOptions[indexOption] + ' ' + pathMainFolder + '/' + arrayPathInputFolders[indexFolder] + '/' + arrayTcks[indexTck] + ' ' + pathMainFolder + '/' + arrayPathInputFolders[indexFolder] + '/' + nameRegAtlasImage + ' ' + pathOutputConnectome
                
                arrayPathOutputConnectomes[(indexFolder + indexOption + indexTck)] = pathOutputConnectome
                cmdArray[(indexFolder + indexOption + indexTck)] = cmd
        
                if printCmd:
                    print(cmd)
            
                if executeCmd:
                    !{cmd}
                
        
    
    return arrayPathOutputConnectomes,cmdArray


/home/alvesrod/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [1]:
print("Start")
img = nib.load('/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/atlasesTemplates/p56_original_smallRegionsGrouped/p56_smallRegionsGrouped_v5_leftRight.nii')
affine = img.affine
data = img.get_data()
labels = np.unique(data)
number_of_labels = len(labels)
labels = np.unique(data).astype('int16')
#size = data.shape + (10,)   # add dimension for labels
#labels = [int(x) for x in input().split()] # get int input from user
print(labels, len(labels))
data4d = np.zeros((data.shape[0], data.shape[1], data.shape[2], len(labels)-1))
num_cores = 8
for idx, i in enumerate(labels-1):
    if (idx==0):
        continue
    print(i+1,idx)
    data1 = np.copy(data)
    data1[data1!=i+1]=0
    data1[data1==i+1]=1
    data4d[:,:,:,idx-1]=data1
    data4d = data4d.astype(np.int16)
    
test_img = nib.Nifti1Image(data4d, affine)
test_img.set_data_dtype(np.int16)
save_name = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/atlasesTemplates/p56_original_smallRegionsGrouped/4dAtlasForConnectivityMatrix/p56_smallRegionsGrouped_v5_leftRight_4D.nii'   
nib.save(test_img, save_name)
print("It worked")
# XXX - Verify if all regions are really present

Start


<ipython-input-1-ccdc224a18eb>:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()


[   0    7   19   45  108  165  227  361  379  454  495  555  573  575
  587  593  608  644  717  809  816  822  885  937 1020 1067 1101 1104
 1106 1108 1109 1111 1112 1114 1115 1116 1117 1119 1120 1121 1122 1126
 1127 1129 1130 1132 1135 1137 1139 1140 1141 1142 1145 1156 1159 1172
 1174 1175 1177 1178 1179 1180 1181 1188 1189 1192 1193 1194 1195 1196
 1198 1199 1200 1201 1202 1207 1208 1212 1216 1217 1221 1227 1228 1229
 1230 1231 1240 1242 1243 1245 1246 1247 1251 1252 1254 1255 1259 1260
 1262 1265 1278 1279 1280 1281 1285 1286 1287 1293 1295 1296 1299 1300
 1301 1302 1303 2007 2019 2045 2108 2165 2227 2361 2379 2454 2495 2555
 2573 2575 2587 2593 2608 2644 2717 2809 2816 2822 2885 2937 3020 3067
 3101 3104 3106 3108 3109 3111 3112 3114 3115 3116 3117 3119 3120 3121
 3122 3126 3127 3129 3130 3132 3135 3137 3139 3140 3141 3142 3145 3156
 3159 3172 3174 3175 3177 3178 3179 3180 3181 3188 3189 3192 3193 3194
 3195 3196 3198 3199 3200 3201 3202 3207 3208 3212 3216 3217 3221 3227
 3228 

In [2]:

# ##################################################################################################################
# Main code
# ##################################################################################################################

# ******************************************************************************************************************
# Variables 
# ******************************************************************************************************************

pathMainInputDir = '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5'

#pathAtlas4D = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/atlasesTemplates/p56_original_smallRegionsGrouped/p56_smallRegionsGrouped_v5_leftRight.nii' # For 3D processing
pathAtlas4D = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/atlasesTemplates/p56_original_smallRegionsGrouped/4dAtlasForConnectivityMatrix/p56_smallRegionsGrouped_v5_leftRight_4D.nii' # For 4D processing


#pathOutput = '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230314_testRegisteringRabies_anatDTI_invTrans/atlas3D' # For 3D processing
pathOutput = '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D' # For 4D processing

arraySubjects = ['M672','M872','M874','M875','M886','M8876','M757','M857','M866','M867b','M877','M881b']


cmd_generalTransAppli = 'antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear'
# ******************************************************************************************************************
# ******************************************************************************************************************
# ******************************************************************************************************************







In [3]:
# ******************************************************************************************************************
# Execution
# ******************************************************************************************************************

os.chdir(pathOutput)

# ==================================================================================================================
# Resample atlas to the resolution of the EPI
# ==================================================================================================================

pathAtlas4D_fileFolder, pathAtlas4D_fileName = os.path.split(pathAtlas4D)
#print(pathAtlas4D_fileFolder)
#print(pathAtlas4D_fileName)

pathOutputResampledAtlas = pathOutput + '/' + pathAtlas4D_fileName[:-4] + '_resampled.nii'

cmd_resampledAtlas = cmd_generalTransAppli + ' -t identity' + \
' --input ' + pathAtlas4D + \
' --output ' +  pathOutputResampledAtlas + \
' --reference-image ' + pathMainInputDir + '/' + 'main_wf/resample_template/resampled_template.nii.gz'
print(cmd_resampledAtlas)


#!{cmd_resampledAtlas}

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


# ==================================================================================================================
# Apply inverse warp and affine transformation template to common space
# ==================================================================================================================

pathOutputResampledAtlas_tmpl2ComInvWarp = pathOutputResampledAtlas[:-4] + '_template2CommonInverseWarp.nii'

cmd_resampledAtlas_tmpl2ComInvWarp = cmd_generalTransAppli + ' --input ' + pathOutputResampledAtlas + ' --output ' + pathOutputResampledAtlas_tmpl2ComInvWarp + ' --reference-image ' + pathMainInputDir + '/commonspace_datasink/warped_template/secondlevel_template0_output_warped_image.nii.gz' + ' --transform ' + pathMainInputDir + '/transforms_datasink/template_to_common_inverse_warp/secondlevel_template0_output_1InverseWarp.nii.gz'


pathOutputResampledAtlas_tmpl2ComInvWarpAff = pathOutputResampledAtlas_tmpl2ComInvWarp[:-4] + 'Affine.nii'
cmd_resampledAtlas_tmpl2ComInvWarpAff = cmd_generalTransAppli + ' --input ' + pathOutputResampledAtlas_tmpl2ComInvWarp + ' --output ' + pathOutputResampledAtlas_tmpl2ComInvWarpAff + ' --reference-image ' + pathMainInputDir + '/commonspace_datasink/warped_template/secondlevel_template0_output_warped_image.nii.gz' + ' --transform [' + pathMainInputDir + '/transforms_datasink/template_to_common_affine/secondlevel_template0_output_0GenericAffine.mat' + ',1]'

#!{cmd_resampledAtlas_tmpl2ComInvWarp}
#!{cmd_resampledAtlas_tmpl2ComInvWarpAff}


print(cmd_resampledAtlas_tmpl2ComInvWarp)
print(cmd_resampledAtlas_tmpl2ComInvWarpAff)
# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


# ==================================================================================================================
# Application of anat2Template transformations (inverse warp and inverse affine)
# ==================================================================================================================

# -----------------------------------------------------------------------------------------------------------------
# Anat to template - References (for inverse warp and inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkRef = pathMainInputDir + '/anat_datasink/anat_labels'

refArrayPathImage = BuildImagePath(pathDatasinkRef,0,'',0)
#PrintArray(refArrayPathImage)

refArrayPathImageOrg = SortArrayByRefArray(refArrayPathImage, arraySubjects, 0)
#PrintArray(refArrayPathImageOrg)


# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Transform (inverse warp)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/anat_to_template_inverse_warp'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,1,'',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Execution (inverse warp)
# -----------------------------------------------------------------------------------------------------------------
atlasResampledPath_tmpl2CommInvWarpAffine = pathOutputResampledAtlas_tmpl2ComInvWarpAff

arrayAtlasResampledPath_tmpl2CommInvWarpAffine = [atlasResampledPath_tmpl2CommInvWarpAffine]*len(arraySubjects)
#PrintArray(atlasResampledPath_tmpl2CommInvWarpAffineArray)

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine, refArrayPathImageOrg, transArrayPathImageOrg, '_anat2TemplateInverseWarp', '.nii', 1, 4, 0, 0)


PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp)
PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Transform (inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/anat_to_template_affine'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,1,'',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Execution (inverse affine)
# -----------------------------------------------------------------------------------------------------------------


arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarpAffine, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarpAffine = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp, refArrayPathImageOrg, transArrayPathImageOrg, 'Affine', '.mat', 1, 9, 0, 0)


PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarpAffine)
PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarpAffine)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


# ==================================================================================================================
# Application of bold2anat transformations (inverse warp and inverse affine)
# ==================================================================================================================

# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - References (for inverse warp and inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkRef = pathMainInputDir + '/bold_datasink/corrected_bold_ref'

refArrayPathImage = BuildImagePath(pathDatasinkRef,0,'/_run_None',0)
#PrintArray(refArrayPathImage)

refArrayPathImageOrg = SortArrayByRefArray(refArrayPathImage, arraySubjects, 0)
#PrintArray(refArrayPathImageOrg)


# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Transform (inverse warp)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/inverse_warp_bold2anat'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,0,'/_run_None',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Execution (inverse warp)
# -----------------------------------------------------------------------------------------------------------------

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarp = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarpAffine, refArrayPathImageOrg, transArrayPathImageOrg, '_bold2AnatInverseWarp', '.nii', 1, 9, 0, 0)


PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarp)
PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------



# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Transform (inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/affine_bold2anat'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,0,'/_run_None',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Execution (inverse affine)
# -----------------------------------------------------------------------------------------------------------------


arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarpAffine, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarpAffine = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp, refArrayPathImageOrg, transArrayPathImageOrg, 'Affine', '.mat', 1, 9, 0, 0)


#PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarpAffine)
#PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarpAffine)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================

# ******************************************************************************************************************
# ******************************************************************************************************************
# ******************************************************************************************************************

antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear -t identity --input /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/atlasesTemplates/p56_original_smallRegionsGrouped/4dAtlasForConnectivityMatrix/p56_smallRegionsGrouped_v5_leftRight_4D.nii --output /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled.nii --reference-image /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5/main_wf/resample_template/resampled_template.nii.gz
antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear --input /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_o

Array size: 12 items.
Array with paths made!!!



Printing array...
Array length: 12
antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear --input /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarp_M672.nii --output /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_M672.nii --reference-image /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5/anat_datasink/

Array size: 12 items.
Array with paths made!!!



Creating array of image paths...
Array size: 12 items.
Array with paths made!!!



Printing array...
Array length: 12
antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear --input /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_M672.nii --output /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_M672.nii --reference-image /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/pos

Array size: 12 items.
Array with paths made!!!




In [4]:
# ==================================================================================================================
# Generation and application of correctedBold2inputBold transformations (composite) on registered atlas for corrected bold 
# ==================================================================================================================

# ------------------------------------------------------------------------------------------------------------------
# Create path array of input EPI images
# ------------------------------------------------------------------------------------------------------------------
pathInputBold = pathMainInputDir + '/bold_datasink/input_bold'

pathInputBoldImage = BuildImagePath(pathInputBold,1,'',0)
#PrintArray(pathInputBoldImage)
pathInputBoldImageOrg = SortArrayByRefArray(pathInputBoldImage, arraySubjects, 0)
#PrintArray(pathInputBoldImageOrg)
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------



# ------------------------------------------------------------------------------------------------------------------
# Create path array of corrected EPI images (effectively used by RABIES)
# ------------------------------------------------------------------------------------------------------------------
pathCorrectedBold = pathMainInputDir + '/bold_datasink/corrected_bold'

pathCorrectedBoldImage = BuildImagePath(pathCorrectedBold,0,'/_run_None',0)
#PrintArray(pathCorrectedBoldImage)
pathCorrectedBoldImageOrg = SortArrayByRefArray(pathCorrectedBoldImage, arraySubjects, 0)
#PrintArray(pathCorrectedBoldImageOrg)

# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------



# ------------------------------------------------------------------------------------------------------------------
# Calculate mean of input and corrected EPI for further calculation
# ------------------------------------------------------------------------------------------------------------------
# Original command: "fslmaths /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/corrected_bold/_scan_info_subject_idF572.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_combined.nii.gz -Tmean ./correctedBold_meansOverTime.nii.gz"

arrayPath_inputBoldMeanOverTime, arrayCmd_inputBoldMeanOverTime = TapFSLTerminalCommand('fslmaths', arraySubjects, pathInputBoldImageOrg, '-Tmean', 'inputBold_meanOverTime', pathOutput, 0, 0)
#PrintArray(arrayPath_inputBoldMeanOverTime)

# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------------------------
# Calculate mean of input and corrected EPI for further calculation
# ------------------------------------------------------------------------------------------------------------------
arrayPath_correctedBoldMeanOverTime, arrayCmd_correctedBoldMeanOverTime = TapFSLTerminalCommand('fslmaths', arraySubjects, pathCorrectedBoldImageOrg, '-Tmean', 'correctedBold_meanOverTime', pathOutput, 0, 0)
#PrintArray(arrayPath_correctedBoldMeanOverTime)
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------



# ------------------------------------------------------------------------------------------------------------------
# Calculate correctedEPI2inputEPI registering transformation
# ------------------------------------------------------------------------------------------------------------------
# Original command: "antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initialize-transforms-per-stage 0 --interpolation Linear --output [ outputMeanOverTime_, outputMeanOverTime_warped_image.nii.gz ] --save-state trans.mat --transform Affine[ 2.0 ] --metric Mattes[ ./inputBold_meanOverTime.nii.gz, ./correctedBold_meanOverTime.nii.gz, 1, 32, Random, 0.05 ] --convergence [ 1500x200, 1e-08, 20 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --transform SyN[ 0.25, 3.0, 0.0 ] --metric Mattes[ ./inputBold_meanOverTime.nii.gz, ./correctedBold_meanOverTime.nii.gz, 1, 32 ] --convergence [ 100x50x30, 1e-09, 20 ] --smoothing-sigmas 2.0x1.0x0.0vox --shrink-factors 3x2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --winsorize-image-intensities [ 0.0, 1.0 ]  --write-composite-transform 1"


arrayPath_correctedBold2InputBold_regImage, arrayPath_correctedBold2InputBold_regTransMat, arrayPath_correctedBold2InputBold_regCompositeT, arrayPath_correctedBold2InputBold_regInvCompositeT, arrayCmd_correctedBold2InputBold = TapANTsRegisteringTerminalCommand('antsRegistration -v --collapse-output-transforms 0 --dimensionality 3 --initialize-transforms-per-stage 0 --interpolation Linear', arraySubjects, arrayPath_inputBoldMeanOverTime, arrayPath_correctedBoldMeanOverTime, pathOutput, 7, 0, 0)
#PrintArray(arrayPath_correctedBold2InputBold_regCompositeT)
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------



# ------------------------------------------------------------------------------------------------------------------
# Apply correctedEPI2inputEPI registering transformation on manually registered atlas (4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp.nii)
# ------------------------------------------------------------------------------------------------------------------

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarp_correctedBold2InputBold = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp, arrayPath_inputBoldMeanOverTime, arrayPath_correctedBold2InputBold_regCompositeT, '_corrBold2InputBold', '.mat', 0, 9, 0, 0)

PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold)
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


Creating array of image paths...
Array size: 12 items.
Array with paths made!!!



Creating array of image paths...
Array size: 12 items.
Array with paths made!!!



Printing array...
Array length: 12
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M672.nii
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M872.nii
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/2

In [6]:
# ==================================================================================================================
# Verify main paths before calculating connectivity matrices
# ==================================================================================================================

PrintArray(pathInputBoldImageOrg)

#PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold)

#PrintArray(arrayPath_brainMask4InputBold)



# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


Printing array...
Array length: 12
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5/bold_datasink/input_bold/M672-2018-12-10082106_DTI3D_mrLinTrans_fslSwapDim-RAS_b0s_nRepB0-5_bold.nii.gz
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5/bold_datasink/input_bold/M872-2018-12-12095919_DTI3D_mrLinTrans_fslSwapDim-RAS_b0s_nRepB0-5_bold.nii.gz
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5/bold_datasink/input_bold/M874-2019-01-29083240_DTI3D_mrLinTrans_fslSwapDim-RAS_b0s_nRepB0-5_bold.nii.gz
/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/rabiesDTI2AnatRegistering/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5/bold_datasin

In [7]:
# ******************************************************************************************************************
# Create structural connectomes
# ******************************************************************************************************************
#XXX
# ==================================================================================================================
# Use 3D reg. atlas organized by Matlab and match frame of current regitered atlas to original image frame
# ==================================================================================================================

suffixConv4D_2_3DViaMatlab_origLabels = "_conv3D_originalLabels_minThres-0.1.nii"
arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim, arrayCmdarrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim = TapFSLSwapDimTerminalCommand('-x -y z', arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold, 4, suffixConv4D_2_3DViaMatlab_origLabels, '_fslSwapDim', '.nii.gz', 0, 1)

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim_mrLinT, arrayCmdarrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim_mrLinT = TapMrTransformTerminalCommand('-linear', arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim, 7, '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/codes/correctFrameAndRAS/linearTransform_3dDTI.txt', '_mrLinTrans', '.nii', 0, 1)


suffixConv4D_2_3DViaMatlab_contLabels = "_conv3D_continuousLabels_minThres-0.1.nii"
arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_contLabels_fslSwapDim, arrayCmdarrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim = TapFSLSwapDimTerminalCommand('-x -y z', arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold, 4, suffixConv4D_2_3DViaMatlab_contLabels, '_fslSwapDim', '.nii.gz', 0, 1)

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_contLabels_fslSwapDim_mrLinT, arrayCmdarrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim_mrLinT = TapMrTransformTerminalCommand('-linear', arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_contLabels_fslSwapDim, 7, '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/codes/correctFrameAndRAS/linearTransform_3dDTI.txt', '_mrLinTrans', '.nii', 0, 1)
# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


# ==================================================================================================================
# Match registered atlas to tractography folders
# ==================================================================================================================

# ------------------------------------------------------------------------------------------------------------------
# Labels with original numbering 
# ------------------------------------------------------------------------------------------------------------------
pathTratography = "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6"
arrayOrderSubjTractog2RegAtlas = ["Mouse-1","Mouse-2","Mouse-3","Mouse-4","Mouse-5","Mouse-6","Mouse-7","Mouse-8","Mouse-9","Mouse-10","Mouse-11","Mouse-12"]
nameRegAtlasTracFolder_origLabels = "registeredAtlas_originalLabels.nii"


arrayRegAtlasInTracFolder_origLabels, arrayCmdRegAtlasInTracFolder_origLabels = TapCopyTerminalCommandReg3DAtlas2TractFolder('cp -v', arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_origLabels_fslSwapDim_mrLinT, 4, pathTratography, arrayOrderSubjTractog2RegAtlas, nameRegAtlasTracFolder_origLabels, 0, 1)
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------


# ------------------------------------------------------------------------------------------------------------------
# Labels with continuous numbering
# ------------------------------------------------------------------------------------------------------------------
pathTratography = "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6"
arrayOrderSubjTractog2RegAtlas = ["Mouse-1","Mouse-2","Mouse-3","Mouse-4","Mouse-5","Mouse-6","Mouse-7","Mouse-8","Mouse-9","Mouse-10","Mouse-11","Mouse-12"]
nameRegAtlasTracFolder_contLabels = "registeredAtlas_continuousLabels.nii"

arrayRegAtlasInTracFolder_contLabels, arrayCmdRegAtlasInTracFolder_contLabels = TapCopyTerminalCommandReg3DAtlas2TractFolder('cp -v', arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp_correctedBold2InputBold_contLabels_fslSwapDim_mrLinT, 4, pathTratography, arrayOrderSubjTractog2RegAtlas, nameRegAtlasTracFolder_contLabels, 0, 1)
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------------------

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================

mrtransform: [ 56%] uncompressing image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M672_conv3D_originalLabels_minThres-0.1_fslSwapDim.nii.gz"...

mrtransform: [100%] uncompressing image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M672_conv3D_originalLabels_minThres-0.1_fslSwapDim.nii.gz"


mrtransform: [100%] preloading data for "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M672_conv3D_originalLabels_minThres-0.1_fslSwapDim.nii.gz"


mrtransform: [100%] copying from "/data_netw...Thres-0.1_fslSwapDim.nii.gz" to "/data_netw...1_fslSwapDim_mrLinTrans.nii"...[0K
mrtransform: [100%] uncompressing image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M872_conv3D_originalLabels_minThres-0.1_fslSwapDim.nii.gz"[0K
mrtransform: [100%] preloading data for "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M872_conv3D_originalLabels_minThres-0.1_fslSwapDim.n

mrtransform: [100%] copying from "/data_netw...Thres-0.1_fslSwapDim.nii.gz" to "/data_netw...1_fslSwapDim_mrLinTrans.nii"...[0K
mrtransform: [100%] uncompressing image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarp_Affine__bold2AnatInverseWarp__corrBold2InputBold_M867b_conv3D_originalLabels_minThres-0.1_fslSwapDim.nii.gz"[0K
mrtransform: [100%] preloading data for "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarp_Affine__bold2AnatInverseWarp__corrBold2InputBold_M867b_conv3D_originalLabels_minThres-0.1_fslS

mrtransform: [100%] preloading data for "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M886_conv3D_continuousLabels_minThres-0.1_fslSwapDim.nii.gz"[0K
mrtransform: [100%] copying from "/data_netw...Thres-0.1_fslSwapDim.nii.gz" to "/data_netw...1_fslSwapDim_mrLinTrans.nii"...[0K
mrtransform: [100%] uncompressing image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarp_Affine__bold2AnatInverseWarp__corrBold2InputBold_M8876_conv3D_continuousLabels_minThres-0.1_fslS

'/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M872_conv3D_originalLabels_minThres-0.1_fslSwapDim_mrLinTrans.nii' -> '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-2/registeredAtlas_originalLabels.nii'
'/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M874_conv3D_originalLabels_minThres-0.1_

'/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M875_conv3D_continuousLabels_minThres-0.1_fslSwapDim_mrLinTrans.nii' -> '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-4/registeredAtlas_continuousLabels.nii'
'/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/atlas2DTI_inverseTransform/20230526_output_manipDTI-20181210_wt-hetMAP6Mice_nRepB0-5_atlas2DTI-invT/atlas4D/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarp_corrBold2InputBold_M886_conv3D_continuousLabels_minThre

In [9]:
# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================
# Create connectomes
# ==================================================================================================================
# ==================================================================================================================

pathMrtrixFolder = pathTratography

arraySubjectFolders = arrayOrderSubjTractog2RegAtlas

arrayTractograms = ["tracks_3_cutoff-0.2600_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck","tracks_3_cutoff-0.2700_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck","tracks_3_cutoff-0.2800_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck","tracks_3_cutoff-0.2900_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck","tracks_3_cutoff-0.3000_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck","tracks_3_cutoff-0.3100_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck","tracks_3_cutoff-0.3200_minl-0.1_maxl-50_step-0.01_angle-8.1921.tck"]

nameRegisteredAtlas = "registeredAtlas_continuousLabels.nii"
outputPrefixName = "connectome_"

arrayConnOptions = ["-symmetric -zero_diagonal",""]
outputSuffixNameArray = ["_symmetric-zeroDiagonal", "_noOptions" ]

#arrayConnOptions = ["-symmetric -zero_diagonal -scale_invnodevol"]
#outputSuffixNameArray = ["_symmetric-zeroDiagonal-scaleInvnodevol" ]

extConnFile = ".csv"

arrayConnectomes, arrayCmdConnectomes = TapTck2ConnectomeTerminalCommand(arrayConnOptions, pathMrtrixFolder, arraySubjectFolders, arrayTractograms, nameRegisteredAtlas, outputPrefixName, outputSuffixNameArray, extConnFile, 0, 1)

#XXX
# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


# ******************************************************************************************************************
# ******************************************************************************************************************
# ******************************************************************************************************************

tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-1/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 67, 76, 77, 81, 92, 99, 114, 151, 154, 162, 166, 167, 190, 191, 195, 196, 206, 213
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labelconvert, or very poor registration)
tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 41, 86, 146, 152, 155, 181, 186, 222
tck2connectome: [WARNING] (This may indicate 

tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 67, 76, 77, 81, 92, 99, 114, 151, 154, 162, 166, 167, 190, 191, 195, 196, 206, 213
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labelconvert, or very poor registration)
tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 41, 50, 82, 85, 86, 146, 148, 155, 164, 173, 181, 186, 199
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-1/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connecto

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 41, 50, 85, 86, 97, 146, 152, 155, 164, 181, 186, 222
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-1/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 67, 76, 77, 81, 92, 99, 114, 151, 154, 162, 166, 167, 190, 191, 195, 196, 206, 213
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in lab

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 41, 67, 72, 99, 146, 206, 213
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-2/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 76, 77, 81, 82, 92, 114, 151, 154, 162, 166, 167, 181, 190, 191, 195, 196
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labelconvert, or very poor registrat

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 32, 34, 45, 50, 67, 72, 85, 99, 146, 148, 164, 199, 206, 213
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-2/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 76, 77, 81, 82, 92, 114, 151, 154, 162, 166, 167, 181, 190, 191, 195, 196
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in label

...

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 32, 50, 67, 85, 92, 97, 99, 146, 148, 164, 199, 213, 216
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-11/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 62, 76, 77, 81, 82, 114, 151, 154, 162, 166, 167, 176, 190, 191, 195, 196, 206
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in lab

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 32, 41, 50, 67, 72, 92, 97, 99, 148, 164, 181, 186, 213
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-11/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 53, 62, 76, 77, 81, 82, 114, 151, 154, 162, 166, 167, 176, 190, 191, 195, 196, 206
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labe

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 32, 67, 72, 86, 97, 99, 148, 155, 164, 186, 206, 213, 222
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-12/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 77, 81, 82, 92, 114, 151, 154, 162, 166, 181, 191, 195, 196
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labelconvert, or very poo

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 32, 45, 67, 85, 99, 148, 164, 167, 199, 206, 213, 222
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-12/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 77, 81, 82, 92, 114, 151, 154, 162, 166, 181, 191, 195, 196
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labelconvert, or very poor re

tck2connectome: [100%] Constructing connectome[0K
tck2connectome: [WARNING] The following nodes do not have any streamlines assigned:
tck2connectome: [WARNING] 32, 67, 85, 97, 99, 148, 155, 199, 206, 213, 222
tck2connectome: [WARNING] (This may indicate a poor registration)
tck2connectome: Image "/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/tractographyConnectome/20230603_022151_MRtrix-output_studyWt-MAP6Mice_allMice_6wt-6hetMAP6/Mouse-12/registeredAtlas_continuousLabels.nii" stored with floating-point type; need to check for non-integer or negative values
tck2connectome: [100%] Verifying parcellation image[0K
tck2connectome: [WARNING] The following nodes are missing from the parcellation image:
tck2connectome: [WARNING] 37, 40, 48, 52, 77, 81, 82, 92, 114, 151, 154, 162, 166, 181, 191, 195, 196
tck2connectome: [WARNING] (This may indicate poor parcellation image preparation, use of incorrect or incomplete LUT file(s) in labelconvert, or very poor registr